In [52]:
%cd ..

/Users/yehormishchyriak/Desktop/BonhamLab/summer2025/microbiome2function


In [53]:
import pandas as pd
import data_prep_scripts

In [54]:
df = pd.read_csv(data_prep_scripts.FETCHED_DATA)

In [55]:
new_df = data_prep_scripts.process_entries(df)

No preprocessing for 'Entry' — skipping.


# Before pre-processing

In [56]:
cols = df.columns

summary = []
for col in cols:
    non_null = df[col].count()
    unique   = df[col].nunique(dropna=True)
    duplication_rate = (non_null - unique) / non_null if non_null else float("nan")
    summary.append({
        "column": col,
        "dimensionality": unique,
        "% entries sharing it": f"{duplication_rate:.2%}",
        "% non-null": f"{non_null / len(df):.2%}"
    })

summary_df = pd.DataFrame(summary)
print(summary_df)

                                column  dimensionality % entries sharing it  \
0                                Entry             263                0.00%   
1                          Domain [FT]           26047                8.78%   
2                          Domain [CC]             328               50.15%   
3                     Protein families            2714               87.84%   
4   Gene Ontology (molecular function)            6310               83.34%   
5   Gene Ontology (biological process)            2786               89.00%   
6                       Interacts with             367                8.93%   
7                        Function [CC]            4594               56.18%   
8                   Catalytic activity            3273               72.90%   
9                            EC number            2580               89.95%   
10                             Pathway            1529               72.22%   
11                             Rhea ID            14

# After pre-processing

In [57]:
cols = new_df.columns

summary = []
for col in cols:
    non_null = new_df[col].count()
    unique   = new_df[col].nunique(dropna=True)
    duplication_rate = (non_null - unique) / non_null if non_null else float("nan")
    summary.append({
        "column": col,
        "dimensionality": unique,
        "% entries sharing it": f"{duplication_rate:.2%}",
        "% non-null": f"{non_null / len(new_df):.2%}"
    })

summary_new_df = pd.DataFrame(summary)
print(summary_new_df)

                                column  dimensionality % entries sharing it  \
0                                Entry             263                0.00%   
1                          Domain [FT]            3569               87.50%   
2                          Domain [CC]             278               57.75%   
3                     Protein families            2714               87.84%   
4   Gene Ontology (molecular function)            6310               83.34%   
5   Gene Ontology (biological process)            2786               89.00%   
6                       Interacts with             367                8.93%   
7                        Function [CC]            3473               66.87%   
8                   Catalytic activity            1637               86.44%   
9                            EC number            2580               89.95%   
10                             Pathway             769               86.03%   
11                             Rhea ID            14

# Sampling entries for data_preparation unittests' construction

## CODING THE NECESSARY FUNCTIONS (NOTE: WILL BE DOUBLED IN 'data_prep_scripts/')

In [64]:
def sample(df: pd.DataFrame, n_rows: int, of_col: "str", seed: int = 1) -> list:
    mask = df[of_col].notna()
    total = df[mask][of_col]
    subset = total.sample(n=n_rows, random_state=seed)
    return subset

In [71]:
df.set_index("Entry", inplace=True, drop=True)

In [81]:
def extract_examples(df: pd.DataFrame, n_rows: int, seed: int = 1):
    out = dict()
    for col in df.columns:
        sampled = sample(df, n_rows, col, seed)
        sampled = zip(sampled, ["<EXPECTED_OUTPUT>"]*len(sampled))
        out[col] = dict(sampled)
    return out

In [82]:
x = extract_examples(df, 2)

In [83]:
x['Domain [FT]']

{'DOMAIN 54..144; /note="Glycosyl hydrolases family 2 sugar binding"; /evidence="ECO:0000259|Pfam:PF02837"; DOMAIN 224..288; /note="Glycoside hydrolase family 2 immunoglobulin-like beta-sandwich"; /evidence="ECO:0000259|Pfam:PF00703"; DOMAIN 298..489; /note="Glycoside hydrolase family 2 catalytic"; /evidence="ECO:0000259|Pfam:PF02836"; DOMAIN 703..759; /note="DUF4982"; /evidence="ECO:0000259|Pfam:PF16355"; DOMAIN 775..862; /note="Glycoside hydrolase family 2"; /evidence="ECO:0000259|Pfam:PF18565"': '<EXPECTED_OUTPUT>',
 'DOMAIN 165..271; /note="TonB-dependent receptor plug"; /evidence="ECO:0000259|Pfam:PF07715"': '<EXPECTED_OUTPUT>'}

### NOTE: < EXPECTED_OUTPUT > fields will be filled out by hand ^^^ (needed for unittests)

## EXTRACTING SAMPLED FIELDS FOR FURTHER MANUAL PROCESSING

In [84]:
examples = extract_examples(df, 20)

In [85]:
import json

In [86]:
with open("tests.json", "w", encoding="utf-8") as f:
    json.dump(examples, f, ensure_ascii=False, indent=2)

In [88]:
new_df["EC number"]

Entry
NaN                      NaN
NaN                  6.3.1.2
NaN                      NaN
NaN                      NaN
NaN       3.1.11.-; 3.1.13.-
                 ...        
A6LCG5                   NaN
C4FRK5                   NaN
D4MZC0               2.7.1.6
K1TBL3                   NaN
R7QVJ6                   NaN
Name: EC number, Length: 42179, dtype: object